In [2]:
import requests
import pandas as pd 
import numpy as np
import gettoken

In [4]:
API_TOKEN = gettoken.mytoken()
API_TOKEN

'413171104bf7e1e93c7b392251dcb7b9'

-----
### Basic info 

In [5]:
serie_a = 71

---- 
### Main API Request

In [9]:
def api_request(url, API_TOKEN, method='GET', headers=None, params=None, payload=None):
    headers =   {'x-rapidapi-key': API_TOKEN,
                'x-rapidapi-host': 'v3.football.api-sports.io'
                }
    try:
        # Fazer a requisição à API
        response = requests.request(method, url, headers=headers, params=params, data=payload)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            try:
                # Tentar retornar a resposta como JSON
                return response.json()
            except ValueError:
                # Se a resposta não for JSON, retornar como texto
                return response.text
        else:
            # Retornar o código de status e o conteúdo da resposta em caso de erro
            return f"Erro {response.status_code}: {response.text}"

    except requests.RequestException as e:
        # Capturar e exibir exceções de requisições
        return f"Erro na requisição: {str(e)}"

 ----
 ### Times Edição

In [44]:
def get_teams_season(API_TOKEN, league, season): 
    url = f'https://v3.football.api-sports.io/standings?league={league}&season={season}'
    result  = api_request(url, API_TOKEN)
    data = []
    # Iterar sobre os dados da resposta
    for league_info in result['response']:  # Corrigido para acessar a chave 'response'
        standings = league_info['league']['standings'][0]  # Acessar o primeiro grupo de standings
        
        for team_info in standings:
            team_id = team_info['team']['id']
            team_name = team_info['team']['name']
            rank = team_info['rank']
            win = team_info['all']['win']
            draw = team_info['all']['draw']
            lose = team_info['all']['lose']
            
            # Adicionar os dados à lista
            data.append({
                'team_id': team_id,
                'team_name':team_name,
                'team_rank': rank,
                'win': win,
                'draw': draw,
                'loss': lose
            })
    # Criar um DataFrame a partir da lista de dados
    df = pd.DataFrame(data)
    return df

In [45]:
df_teams_season = get_teams_season(API_TOKEN, 71, 2020)
df_teams_season.head(4)

,team_id,team_name,team_rank,win,draw,loss
0,127,Flamengo,1,21,8,9
1,119,Internacional,2,20,10,8
2,1062,Atletico-MG,3,20,8,10
3,126,Sao Paulo,4,18,12,8


----
### Get Fixtures Season

In [54]:
def get_teams_season(API_TOKEN, league, season):
    url = f'https://v3.football.api-sports.io/fixtures?league={league}&season={season}'
    res = api_request(url, API_TOKEN)
    # Criar o DataFrame diretamente com list comprehension
    df = pd.DataFrame([{
        'jogo_id': game['fixture']['id'],
        'mandante_id': game['teams']['home']['id'],
        'visitante_id': game['teams']['away']['id'],
        'round': game['league']['round']
    } for game in res['response']])
    return df

In [57]:
get_teams_season(API_TOKEN, 71, 2017)

,jogo_id,mandante_id,visitante_id,round
0,35097,144,124,Regular Season - 38
1,35098,134,121,Regular Season - 38
2,35099,1062,130,Regular Season - 38
3,35100,120,135,Regular Season - 38
4,35101,132,147,Regular Season - 38
...,...,...,...,...
375,35472,121,133,Regular Season - 1
376,35473,139,123,Regular Season - 1
377,35474,124,128,Regular Season - 1
378,35475,131,132,Regular Season - 1


-----
### Estatísticas do Jogo

In [73]:
def get_fixture_stats(API_TOKEN, fixture):
    url = f'https://v3.football.api-sports.io/fixtures/statistics?fixture={fixture}'
    res = api_request(url,API_TOKEN)
    # Preparar os dados para o DataFrame
    data = []
    for team_data in res['response']:
        team_stats = {'team_id': team_data['team']['id'], 'team_name': team_data['team']['name']}
        for stat in team_data['statistics']:
            team_stats[stat['type']] = stat['value']
        data.append(team_stats)

    # Criar o DataFrame
    df = pd.DataFrame(data)
    df['fixture'] = fixture
    return df

In [74]:
get_fixture_stats(API_TOKEN,35475)

,team_id,team_name,Shots on Goal,Shots off Goal,Total Shots,Blocked Shots,Shots insidebox,Shots outsidebox,Fouls,Corner Kicks,Offsides,Ball Possession,Yellow Cards,Red Cards,Goalkeeper Saves,Total passes,Passes accurate,Passes %,fixture
0,131,Corinthians,2,3,5,0,2,3,10,1,1,66%,2,0,1,638,539,84%,35475
1,132,Chapecoense-sc,2,8,16,6,9,7,12,7,3,34%,2,0,1,304,227,75%,35475
